In [1]:
import pandas as pd
import warnings

In [2]:
pd.set_option('display.max_columns', None)
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=UserWarning)

In [3]:
def create_df():
    # This function creates a simple dataframe with all the info we need from the dataset
    dfs = []
    features = ['ATP', 'Location', 'Tournament', 'Date', 'Series', 'Court', 'Surface',
       'Round', 'Best of', 'Winner', 'Loser', 'WRank', 'LRank', 'WPts', 'LPts',
       'W1', 'L1', 'W2', 'L2', 'W3', 'L3', 'W4', 'L4', 'Comment','B365W','B365L','PSW','PSL']
    for year in range(2000,2026):
        # Pre 2013 data is stored in xls rather than xlsx files
        if year < 2013:
            df_year = pd.read_excel(f"C:\\Users\\olive\\Python Projects\\Tennis Predictions\\data\\{year}.xls")
        else:
            df_year = pd.read_excel(f"C:\\Users\\olive\\Python Projects\\Tennis Predictions\\data\\{year}.xlsx")
        # Some years do not include the following features so we introduce them as NA columns
        for feature in ['WPts', 'LPts','B365W','B365L','PSW','PSL']:
            if feature not in df_year.columns:
                df_year[feature] = pd.NA
        # Some features in the data are not relevant so we exclude them
        df_year = df_year[features]
        dfs.append(df_year)
    df = pd.concat(dfs, ignore_index=True)
    return df

In [4]:
def convert_dates(df):
    # This function converts the date feature into day, month and year features which have integer values
    df['Day'] = pd.to_datetime(df['Date'].values).day
    df['Month'] = pd.to_datetime(df['Date'].values).month
    df['Year'] = pd.to_datetime(df['Date'].values).year
    df = df.drop('Date', axis = 1)
    return df

In [5]:
def create_unbiased_df(df):
    # This function creates a new dataframe which does not assume knowledge of the match result anywhere
    df = df.copy()
    # First we rename all variable names that include reference to the winner or loser
    new_names = {'Winner': 'Player 1', 'Loser': 'Player 2', 'WRank': 'P1 Rank', 'LRank': 'P2 Rank', 
        'WPts': 'P1 Pts', 'LPts': 'P2 Pts', 'W1': 'P1 Set 1', 'L1': 'P2 Set 1', 'W2': 'P1 Set 2',
        'L2': 'P2 Set 2', 'W3': 'P1 Set 3', 'L3': 'P2 Set 3', 'W4': 'P1 Set 4', 'L4': 'P2 Set 4',
        'B365W': 'B365 P1', 'B365L': 'B365 P2', 'PSW': 'PS P1', 'PSL': 'PS P2'}
    df.rename(columns = new_names, inplace = True)
    '''
    Next we order the players alphabetically and switch player-based features accordingly
    We denote the name 'wrong_rows' to all rows where the players are not alread in alphabetical order
    The rows are the only ones that require any switching
    '''
    wrong_rows = df['Player 1'] > df['Player 2']
    P1_features = list(new_names.values())[::2]
    P2_features = list(new_names.values())[1::2]
    temp = df.loc[wrong_rows, P1_features].copy()
    df.loc[wrong_rows, P1_features] = df.loc[wrong_rows, P2_features].values
    df.loc[wrong_rows, P2_features] = temp.values
    return df

In [6]:
def add_target_var(df, unbiased_df):
    # This function produces a dataframe containing the target variable 'Result'
    unbiased_df['Result'] = (df['Winner'] == unbiased_df['Player 1']).astype(int)
    return unbiased_df

In [7]:
df = create_df()
df = convert_dates(df)
unbiased_df = create_unbiased_df(df)
df = add_target_var(df, unbiased_df)
df.to_csv('df.csv', index = False)